# скилы


Подготовка окружения

In [ ]:
!pip install toloka-kit==0.1.6
!pip install pandas
#!pip install ipyplot

#import datetime
#import time

import pandas
#import ipyplot

import toloka.client as toloka
#import toloka.client.project.template_builder as tb

In [ ]:
toloka_client = toloka.TolokaClient(input('Enter you Toloka token:'), 'PRODUCTION')
print(toloka_client.get_requester())

- зачем нужны и как использовать
- встроенные скилы
- собственные скилы


# Динамический расчёт навыка по своей формуле
Допустим есть простейший проект по модерации (т.е. классификации двумя классами) и настроенный пул в нём. И в нём есть задания с голден-сетами. Нам интересно рассчитывать навык который бы отражал precision испонителя.

В проекте есть только ответы "Да" и "Нет". Где "Да" означает принадлежность к искомому классу.

In [ ]:
pool_id = ''
skill_id = ''
pool = toloka_client.open_pool(pool_id)
output_field_name = ''

In [ ]:
class SkillUpdater:
    """
    Args:
        skill_id - идентификатор скила который надо обновлять
        toloka_client - экземпляр клиента, для работы с api
        output_field_name - выходное поле на основании которого считать precision
            оно должно быть в выходных данных и в голден-сете 
    """
    def __init__(self, toloka_client, pool, skill_id, output_field_name):
        self.toloka_client = toloka_client
        self.output_field_name = output_field_name
        self.skill_id = skill_id
        self.last_assignment = None  # когда последний раз вычитывали ответы, чтобы не вычитывать все ответы каждый раз
        # словарь в котором храним данные для расчёта
        # ключ: идентификатор исполнителя
        # значение: словарь {'TP':0, 'TP+FP': 0} для вычисления precision
        self.performers_precision = {}
        self.performers_for_update = set()  # запоминаем по каким исполнителям стоит обновлять скилы

    def process_new_assignments(self):
        ### Обработать новые ответы
        request = toloka.search_requests.AssignmentSearchRequest(
            status=toloka.Assignment.SUBMITTED,
            pool_id=self.from_pool_id,
            submitted_gt = self.last_assignment,
        )
        for assignment in toloka_client.get_assignments(request):
            for task, solution in zip(assignment.tasks, assignment.solutions):
                self.last_assignment = max(self.last_assignment, assignment.submitted) if self.last_assignment else assignment.submitted
                if task.known_solutions is None:
                    continue  # работаем только с годен-сетами, все остальные ответы тут пропускаем
                current_precision = self.performers_precision.setdefault(assignment.user_id, {'TP':0, 'TP+FP': 0})
                current_precision['TP+FP'] += 1
                performer_answer = solution.output_values[self.output_field_name]
                self.performers_for_update.add(assignment.user_id)
                # обычно правильный ответ один, но может быть и несколько
                # в этом случае так же стоит подумать над обработкой known_solution.correctness_weight - вес правильного ответа
                for known_solution in task.known_solutions:
                    if performer_answer == known_solution.output_values[self.output_field_name]
                        current_precision['TP'] += 1
                        break

    def set_skills(self):
        ### Обновить посчитанные скилы в Толоку
        while self.performers_for_update:
            performer_id = self.performers_for_update.pop()
            if performer_id not in self.performers_precision:
                continue  # тут можно как-то обработать или зафиксировать эту странную ситуацию
            current_precision = self.performers_precision[performer_id]
            # можно добавить ограничение, чтобы precision начинался считаться с какого-то минимального количества ответов
            if current_precision['TP+FP'] < 5:
                continue
            skill_value = current_precision['TP'] / current_precision['TP+FP']
            for _ in range(3):
                try:
                    toloka_client.set_user_skill(skill_id=self.skill_id, user_id=performer_id, value=skill_value)
                    break
                except:
                    sleep(60)

In [ ]:
skill_updater = SkillUpdater(
    toloka_client=toloka_client,
    pool=pool,
    skill_id=skill_id,
    output_field_name=output_field_name)

while not pool.is_closed():
    skill_updater.process_new_assignments()
    skill_updater.set_skills()
    sleep(60*10)